In [1]:
import ioutils as io
import skimage
from PIL import Image
import glob
import os
import natsort
import numpy as np
from skimage.transform import resize
import random
import cv2
from PIL import Image
from collections import OrderedDict
import random
import sys
import pickle
import open3d as o3d

from torchvision import transforms
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable

sys.path = ['./cut/'] + sys.path

from my_networks import scr_net, CreateDiscriminator
from models.base_model import BaseModel
from models import networks

from scr_utils import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
norm = 2
epochs = 20
b_size = 48
print_interval = 500
# train_real_proba = 1.0
date = '0320'
NAME = 'scr_cyclegan_b{}_{}_train_his_cut'.format(b_size, date)

data_path = '/scratch/zq415/work/pose/data'


In [3]:
render_img_paths, render_scr_paths, render_poses = make_render_dataset(data_path)
### set the render_img_paths to histogram matched
render_img_paths = natsort.natsorted(glob.glob('/scratch/zq415/work/pose/data/train_render_matched'+'/**/*.png', recursive=True))
print(len(render_img_paths), len(render_scr_paths), render_poses.shape)


real_val_img_paths, val_scr_label_paths, val_poses = make_render_dataset(data_path, train_flag=False)


print(len(real_val_img_paths), len(val_scr_label_paths), val_poses.shape)


100000 100000 (100000, 4, 4)
1637 1637 (1637, 4, 4)


In [4]:
real_val_dataset = dataset_scr(real_val_img_paths, val_poses, val_scr_label_paths,
                              transform=transforms.Compose([fix_crop()]))
real_val_dataloader = DataLoader(real_val_dataset, batch_size=100,
                            shuffle=False, num_workers=4)


render_train_dataset = dataset_scr(render_img_paths, render_poses, render_scr_paths,
                               transform=transforms.Compose([random_crop()]))
render_train_dataloader = DataLoader(render_train_dataset, batch_size=b_size,
                            shuffle=True, num_workers=4)


In [5]:
def train(model, device, train_loader, optimizer, criterion, epoch, record):
    model.train()
    running_loss = 0.0
    for i_batch, sample_batched in enumerate(train_loader):
        inputs, labels = sample_batched['image'], sample_batched['scr']
        inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device, dtype=torch.float)
        
        with torch.no_grad():
            inputs = netG((inputs-0.5)/0.5)*0.5+0.5
            
        # zero the parameter gradients
        optimizer.zero_grad()
        
        outputs = model(inputs)[1]
        loss = criterion(outputs, torch.nn.functional.interpolate(labels, scale_factor=1/8.0, mode='bilinear'),p=norm)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (i_batch+1) % print_interval == 0:
            out_str = "epoch {}, batch {}, current loss {}.\n".format(epoch+1,i_batch,running_loss/print_interval)
            print(out_str)
            record.write(out_str)
            record.flush()
            
            img = inputs.permute(0,2,3,1).cpu().detach().numpy()[0,...]
            label = labels.permute(0,2,3,1).cpu().detach().numpy()[0,...]
            predict = torch.nn.functional.interpolate(outputs,scale_factor=8.0, mode='bilinear').permute(0,2,3,1).cpu().detach().numpy()[0,...]
            print(img.shape)
            running_loss = 0.0
            
def test2(net, dataloader):
    net.eval()
    scr_predicts, true_poses = [], []
    scr_labels = []

    for i_batch, sample_batched in tqdm(enumerate(dataloader)):
        inputs, scrs, poses = sample_batched['image'], sample_batched['scr'], sample_batched['pose']
        inputs = inputs.to(device, dtype=torch.float)
        with torch.no_grad():
            scr_predict = net(inputs)[1]

        scr_predicts.append(scr_predict.cpu().numpy())
        scr_labels.append(torch.nn.functional.interpolate(scrs.cpu(), scale_factor=1/8.0, mode='bilinear').numpy())
        true_poses.append(poses.numpy())
        
    scr_predicts = np.concatenate(scr_predicts)
    scr_labels = np.concatenate(scr_labels)
    true_poses = np.concatenate(true_poses)
    print(scr_predicts.shape, scr_labels.shape, true_poses.shape)
    return scr_predicts, scr_labels, true_poses        



In [6]:
if torch.cuda.is_available():
    device = torch.device(0 if torch.cuda.is_available() else "cpu")
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    print('using GPU for training')
else:
    device = torch.device('cpu')
    print('using CPU for training')
print(device)


using GPU for training
cuda:0


In [7]:
#### sc network
net = scr_net()
    
net.to(device)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
    
model_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(model_total_params)


11572803


In [8]:
###### CUT
netG_device = [i for i in range(torch.cuda.device_count())]
netG = networks.define_G(3, 3, 64, 'resnet_9blocks', 'instance', False, 'xavier', 0.02, False, False, netG_device)

state_dict = torch.load('./cut/checkpoints/match2real_cut/6_net_G.pth', map_location=str(device))
netG.load_state_dict(state_dict)
for param in netG.parameters():
    param.requires_grad = False
netG.eval()


ResnetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Downsample(
      (pad): ReflectionPad2d([1, 1, 1, 1])
    )
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): ReLU(inplace=True)
    (11): Downsample(
      (pad): ReflectionPad2d([1, 1, 1, 1])
    )
    (12): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=

In [9]:
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.00001)
criterion = maskedPnorm #robust_maskedPnorm #maskedPnorm


In [ ]:
record = open('./save_info/'+ NAME +'.txt','w+')

basic_str = "norm: {}, batch_size: {}.\n".format(norm, b_size) 
print(basic_str)
record.write(basic_str)
record.flush()

for epoch in range(epochs):
    
    train(net, device, render_train_dataloader, optimizer, criterion, epoch, record)
    
    real_val_scr_predicts, val_scrs, val_true_poses = test2(net, real_val_dataloader)
    
    out_str = 'real_mean_scr_error: {}.\n'.format(compute_mean_error(val_scrs, real_val_scr_predicts))
    print(out_str)
    record.write(out_str)
    record.flush()
    
    real_val_median = get_median(real_val_scr_predicts, val_true_poses)
    out_str = 'real_val, r_median: {}, t_median: {}.\n'.format(real_val_median[0], real_val_median[1])
    print(out_str)
    record.write(out_str)
    record.flush()
    
    torch.save(net.state_dict(), './check_point/scr_b{}_e{}_norm{}_{}_train_his_cut.pth'.format(b_size,epoch,norm,date))
    
    

norm: 2, batch_size: 48.

epoch 1, batch 499, current loss 1.7265491445064545.

(320, 320, 3)
